In [1]:
# load manipilation_library
import sys
sys.path.append('./manipulation_library.py') # add the path of manipulation_library.py
from manipulation_library import *

In [2]:
def get_center(bbox):
    """
    Calculate the center of a bounding box.
    bbox: [x_min, y_min, x_max, y_max]
    """
    x_center = (bbox[0] + bbox[2]) / 2
    y_center = (bbox[1] + bbox[3]) / 2
    return np.array([x_center, y_center])

def euclidean_distance(point1, point2):
    """
    Calculate the Euclidean distance between two points.
    """
    return np.linalg.norm(point1 - point2)

def find_close_bounding_boxes(ocr_bboxes, seg_bboxes, threshold):
    """
    Find OCR bounding boxes that are close to segmentation bounding boxes.
    ocr_bboxes: List of OCR bounding boxes
    seg_bboxes: List of segmentation bounding boxes
    threshold: Distance threshold to consider bounding boxes as close
    """
    close_bboxes = []
    close_bboxes_indices = []
    for idx_ocr, ocr_bbox in enumerate(ocr_bboxes):
        ocr_center = get_center(ocr_bbox)
        for idx, seg_bbox in enumerate(seg_bboxes):
            seg_center = get_center(seg_bbox)
            distance = euclidean_distance(ocr_center, seg_center)
            if distance <= threshold and ocr_bbox[1] < seg_bbox[1]:
                close_bboxes.append((ocr_bbox, seg_bbox))
                close_bboxes_indices.append((idx, idx_ocr))
    return close_bboxes, close_bboxes_indices

def find_unpaired_seg_bboxes(seg_bboxes, close_bboxes_indices):
    
    """
    Find segmentation bounding boxes that do not have a paired OCR bounding box.
    seg_bboxes: List of segmentation bounding boxes
    close_bboxes_indices: List of indices of close bounding boxes
    """
    paired_seg_indices = {pair[0] for pair in close_bboxes_indices}
    unpaired_seg_indices = [idx for idx in range(len(seg_bboxes)) if idx not in paired_seg_indices]
    return unpaired_seg_indices


In [3]:

# # Example usage
# ocr_bboxes = [[10, 20, 50, 60], [100, 120, 150, 160]]  # Replace with actual OCR bounding boxes
# seg_bboxes = [[12, 22, 52, 62], [200, 220, 250, 260]]  # Replace with actual segmentation bounding boxes
# threshold = 10  # Define your threshold distance

# close_bboxes = find_close_bounding_boxes(ocr_bboxes, seg_bboxes, threshold)
# print(close_bboxes)

In [4]:
main_path = '/home/barradd/Documents/GitHub/CoralSCOP/data/raw/Exp8-CBS-080724'
path_to_coral = f'{main_path}/TL8_2799.jpg'
image = get_image(path_to_coral)
image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE, image)

In [5]:
reader = easyocr.Reader(['en'],gpu=True) # this needs to run only once to load the model into memory

In [6]:
result = reader.readtext(image)
ocr_bboxes, text_list = OcrAnalysis.get_bounding_boxes(result)

In [ ]:
mask_generator = load_sam_model(model_type="vit_b")
masks = mask_generator.generate(image=image)
# list_of_images , titles , image_dataframe = process_images_and_sort_by_coordinates(image = image, masks= masks)


In [8]:
seg_bboxes = [    ]
for i in range(len(masks)):
    x, y, width, height = masks[i]['bbox']
    seg_bboxes.append ( np.array([x, y, x+width, y+height]) ) 

In [ ]:
ocr_bboxes[0:3]

In [ ]:
seg_bboxes[0:3]

In [11]:
threshold = 50  # Define your threshold distance

In [12]:
close_bboxes = find_close_bounding_boxes(ocr_bboxes, seg_bboxes, threshold)

In [ ]:
print(close_bboxes)

In [ ]:
len(close_bboxes)

In [ ]:
for i in range(100,150,5):
    threshold = i
    close_bboxes ,close_bboxes_indices = find_close_bounding_boxes(ocr_bboxes, seg_bboxes, threshold)
    print(f'{threshold} : {len(close_bboxes)} , {close_bboxes_indices}')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Set the threshold
# this seems to be the best threshold for these images
threshold = 125

# Find close bounding boxes with the new threshold
close_bboxes, close_bboxes_indices = find_close_bounding_boxes(ocr_bboxes, seg_bboxes, threshold)

# Create a figure and axis
fig, ax = plt.subplots(1, figsize=(12, 12))

# Display the image
ax.imshow(image)

# Define colors for bounding boxes
colors = plt.get_cmap('tab20', len(close_bboxes))

# Plot OCR and segmentation bounding boxes
for idx, (ocr_bbox, seg_bbox) in enumerate(close_bboxes):
    color = colors(idx)
    
    # OCR bounding box
    rect_ocr = patches.Rectangle((ocr_bbox[0], ocr_bbox[1]), ocr_bbox[2] - ocr_bbox[0], ocr_bbox[3] - ocr_bbox[1], linewidth=2, edgecolor=color, facecolor='none', label=f'OCR {idx}')
    ax.add_patch(rect_ocr)
    
    # Segmentation bounding box
    rect_seg = patches.Rectangle((seg_bbox[0], seg_bbox[1]), seg_bbox[2] - seg_bbox[0], seg_bbox[3] - seg_bbox[1], linewidth=2, edgecolor=color, facecolor='none', linestyle='dashed', label=f'Seg {idx}')
    ax.add_patch(rect_seg)

# Add legend
handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
# ax.legend(by_label.values(), by_label.keys())

# Show the plot
plt.show()
# save the plot
fig.savefig('../data/final/bounding_boxes.png', dpi=300)

In [17]:
# sort masks in the same order as close_bboxes 
sorted_masks = []
for idx, _ in close_bboxes_indices:
    sorted_masks.append(masks[idx])



In [18]:
def background_to_black ( image, index , masks  ):
    # Apply the mask to the image
    masked_img = image.copy()
    masked_pixels = masked_img[masks[index]['segmentation']==True]
    masked_img[masks[index]['segmentation']==False] = (0, 0, 0)  # Set masked pixels to black
    return masked_img ,masked_pixels


def use_sorted_mask(image, masks):
    cropped_image_list = []
    for i in range(len(masks)):
        x, y, width, height = masks[i]['bbox']
        image_b, masked_pixels = background_to_black(image=image, index=i , masks=masks)
        cropped_image = image_b[int(y):int(y+height), int(x):int(x+width)]
        cropped_image_list.append(cropped_image)

    return cropped_image_list

In [19]:
def process_images_and_use_sorted_mask(image, masks):
    cropped_image_list  = use_sorted_mask( image=image , masks=masks )
    return cropped_image_list

In [20]:
cropped_image_list = process_images_and_use_sorted_mask(image, sorted_masks)

In [21]:
# now lets output the images to ../data/interrim/Exp8-CBS-080724 , 
# the name of the image will be the same as the original image with the index of the sorted_masks appended to it
# the name also must include the number of the ocr bounding box that is close to the segmentation bounding box
# the image will be saved as a jpg file
for idx, image_segment in enumerate( cropped_image_list) :
    index_segmetation , index_ocr = close_bboxes_indices[idx]
    pred_text = text_list[index_ocr]
    image_segment = cv2.cvtColor(image_segment, cv2.COLOR_BGR2RGB)
    cv2.imwrite(f'../data/interim/image_index_{index_ocr}_{index_segmetation}_tag_{pred_text}.jpg',image_segment)

In [22]:
# find the segmentation bounding boxes that do not have a paired OCR bounding box
unpaired_seg_indices = find_unpaired_seg_bboxes(seg_bboxes, close_bboxes_indices)
# get the images for the unpaired segmentation bounding boxes
unpaired_images = []
for idx in unpaired_seg_indices:
    x, y, width, height = seg_bboxes[idx]
    image_b, masked_pixels = background_to_black(image=image, index=idx , masks=masks)
    cropped_image = image_b[int(y):int(y+height), int(x):int(x+width)]
    unpaired_images.append(cropped_image)

# now lets output the images to ../data/interrim/ , lets add tag "unpaired" to the name of the image
# the image will be saved as a jpg file
for idx, image_segment in enumerate( unpaired_images) :
    image_segment = cv2.cvtColor(image_segment, cv2.COLOR_BGR2RGB)
    cv2.imwrite(f'../data/interim/image_unpaired_{idx}.jpg',image_segment)